<a href="https://colab.research.google.com/github/SMTorg/smt/blob/master/tutorial/SMT_DesignSpace_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="jumbotron text-left"><b>
    
This tutorial describes how to use de DesignSpace within the SMT toolbox. 
<div>
    
    May 2024 - `SMT version 2.5.1`
  
    Jasper Bussemaker (DLR), Paul Saves, and Nathalie BARTOLI (ONERA/DTIS/M2CI)

<div class="alert alert-info fade in" id="d110">
<p>Some updates</p>
<ol> -  Manipulation of mixed DOE (continuous, integer,  categorical and hierarchical variables) </ol>
</div>

<p class="alert alert-success" style="padding:1em">
To use SMT models, please follow this link : https://github.com/SMTorg/SMT/blob/master/README.md. The documentation is available here: http://smt.readthedocs.io/en/latest/
</p>

The reference paper is available 
here https://www.sciencedirect.com/science/article/pii/S0965997818309360?via%3Dihub 

or as a preprint: http://mdolab.engin.umich.edu/content/python-surrogate-modeling-framework-derivatives

For mixed integer with continuous relaxation, the reference paper is available here https://www.sciencedirect.com/science/article/pii/S0925231219315619

In [1]:
# to install smt
!pip install smt

<div class="alert alert-warning" >
If you use hierarchical variables and the size of your doe greater than 30 points, you may leverage the `numba` JIT compiler to speed up the computation
To do so:
    
 - install numba library
    
     `pip install numba`
    
    
 - and define the environment variable `USE_NUMBA_JIT = 1` (unset or 0 if you do not want to use numba) 
    
     - Linux: export USE_NUMBA_JIT = 1
    
     - Windows: set USE_NUMBA_JIT = 1

</div>

In [2]:
from smt.utils.design_space import (
    DesignSpace,
    FloatVariable,
    IntegerVariable,
    OrdinalVariable,
    CategoricalVariable,
)

# Manipulate DOE with mixed, categorical & hierarchical variables

4 variables 
 - 1 categorical variable with 2 labels ['A', 'B'] # x0 categorical: A or B; order is not relevant
 - 1 ordinal variable with 3 levels ['C', 'D', 'E']),  # x1 ordinal: C, D or E; order is relevant
 - 1 integer variable [0,2]: 3 possibilities: 0, 1, 2
 - 1 continuous variable $\in [0, 1]$
 
 
 **Posssibility to have hierarchical variable: x1 exists only if x0 = 'A'**

In [3]:
# Instantiate the design space with all its design variables:

ds = DesignSpace(
    [
        CategoricalVariable(
            ["A", "B"]
        ),  # x0 categorical: A or B; order is not relevant
        OrdinalVariable(["C", "D", "E"]),  # x1 ordinal: C, D or E; order is relevant
        IntegerVariable(0, 2),  # x2 integer between 0 and 2 (inclusive): 0, 1, 2
        FloatVariable(0, 1),  # c3 continuous between 0 and 1
    ]
)

print("Number of design variables", len(ds.design_variables))
# You can define decreed variables (conditional activation):
ds.declare_decreed_var(
    decreed_var=1, meta_var=0, meta_value="A"
)  # Activate x1 if x0 == A

Number of design variables 4


In [4]:
## To give some examples
# It is also possible to randomly sample design vectors conforming to the constraints:
n = 5
x_sampled, is_acting_sampled = ds.sample_valid_x(5)

print("Data encoded: \n", x_sampled)
print("Data in initial space: \n", ds.decode_values(x_sampled))

Data encoded: 
 [[0.         2.         1.         0.50141714]
 [1.         0.         1.         0.48517298]
 [0.         1.         2.         0.83539973]
 [0.         0.         1.         0.65900747]
 [0.         2.         1.         0.5664969 ]]
Data in initial space: 
 [['A', 'E', 1.0, 0.5014171417556681], ['B', 'C', 1.0, 0.48517297677504534], ['A', 'D', 2.0, 0.835399731580032], ['A', 'C', 1.0, 0.6590074713239295], ['A', 'E', 1.0, 0.5664968989411066]]


In [5]:
# After defining everything correctly, you can then use the design space object
# to correct design vectors and get information about which design variables are acting:
x_corr, is_acting = ds.correct_get_acting(x_sampled)
print("Which variables are active \n", is_acting)

Which variables are active 
 [[ True  True  True  True]
 [ True False  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


In [6]:
# If needed, it is possible to get the legacy design space definition format:
xlimits = ds.get_x_limits()
cont_bounds = ds.get_num_bounds()
unfolded_cont_bounds = ds.get_unfolded_num_bounds()
print("Limits of each variable \n", xlimits)
print("Continuous bounds with the encoding done (4 variables now) \n", cont_bounds)
print(
    "Continuous bounds with the unfolded encoding done (5 variables now)\n",
    unfolded_cont_bounds,
)

Limits of each variable 
 [['A', 'B'], ['0', '1', '2'], (0, 2), (0, 1)]
Continuous bounds with the encoding done (4 variables now) 
 [[0 1]
 [0 2]
 [0 2]
 [0 1]]
Continuous bounds with the unfolded encoding done (5 variables now)
 [[0. 1.]
 [0. 1.]
 [0. 2.]
 [0. 2.]
 [0. 1.]]


# Manipulate DOE with continuous variables

In [7]:
# You can also instantiate a purely-continuous design space from bounds directly:
continuous_design_space = DesignSpace([(0, 1), (0, 2), (0.5, 5.5)])
print(
    "Number of design variables =",
    continuous_design_space.n_dv,
    " or ",
    len(continuous_design_space.design_variables),
)

Number of design variables = 3  or  3


In [8]:
x_sampled_cont, is_acting_sampled_cont = continuous_design_space.sample_valid_x(5)

In [9]:
print("Data encoded: \n", x_sampled_cont)
print("Is_acting: \n", is_acting_sampled_cont)

Data encoded: 
 [[0.31711655 0.1616406  2.58306429]
 [0.49960756 0.62928794 2.96869517]
 [0.11611917 0.63034393 2.74785997]
 [0.44327097 0.3821734  3.78969088]
 [0.32158259 1.88724976 1.06429242]]
Is_acting: 
 [[ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]]
